In [1]:
#conexiones a aura
from neo4j import GraphDatabase
uri = "neo4j+ssc://1e48c053.databases.neo4j.io"
auth= ("neo4j", "R_ro7G4XS1ixvmtnxHX5_Kb0pXxZ_Lohg91TPLoaHZQ")

driver = GraphDatabase.driver(uri=uri, auth=auth)

In [2]:
# Función para ejecutar una consulta en Neo4j
def execute_query(query, parameters={}):
    with driver.session() as session:
        return session.run(query, parameters).data()

In [3]:
def gestion_Relaciones():
    print("Que opcion desea?")
    print("1) Agregar propiedad a relacion especifica")
    print("2) Agregar propiedad a multiples relaciones")
    print("3) Actualizacion de propiedad de relacion especifica")
    print("4) Actualizacion de multiples propiedades")
    print("5) Eliminacion de una propiedad de relacion especifica")
    print("6) Eliminacion de varios propiedades")
    print("7) Salir")
    respuesta=int(input("INGRESE SU RESPUESTA: "))
    return respuesta



In [4]:

def add_empty_property_to_relationship(driver):
    print("Agregar una nueva propiedad a una relación específica")
    
    # Solicitar y validar la entrada para las etiquetas de los nodos y la relación
    relacion = input("Ingrese el tipo de relación que quiere editar (ej. ACTED_IN): ").strip()
    nodoA = input("Ingrese la etiqueta del nodo de salida (ej. Actor): ").strip()
    nodoB = input("Ingrese la etiqueta del nodo de llegada (ej. Movie): ").strip()
    propiedad = input("Ingrese el nombre de la nueva propiedad (ej. role): ").strip()
    
    # Establecer un valor predeterminado seguro
    default_value = ""  # Asumiendo que quieres establecer la propiedad a una cadena vacía

    query = f"""
    MATCH (a:`{nodoA}`)-[r:`{relacion}`]->(b:`{nodoB}`)
    SET r.`{propiedad}` = "{default_value}"
    RETURN count(r) as updatedCount
    """

    # Ejecutar la consulta
    with driver.session() as session:
        result = session.run(query)
        updated_count = result.single()[0]
        print(f"Propiedades actualizadas con el valor por defecto '{default_value}': {updated_count}")


In [5]:


def remove_property_from_relationshipGENERAL(driver):
    print("Eliminar una propiedad de una relación específica")
    
    # Solicitar y validar la entrada para las etiquetas de los nodos y la relación
    relacion = input("Ingrese el tipo de relación de la que desea eliminar una propiedad: ").strip()
    nodoA = input("Ingrese la etiqueta del nodo de salida (ej. Actor): ").strip()
    nodoB = input("Ingrese la etiqueta del nodo de llegada (ej. Movie): ").strip()
    propiedad = input("Ingrese el nombre de la propiedad que desea eliminar (ej. role): ").strip()

    # Construir y ejecutar la consulta Cypher utilizando interpolación segura para nombres
    query = f"""
    MATCH (a:`{nodoA}`)-[r:`{relacion}`]->(b:`{nodoB}`)
    REMOVE r.`{propiedad}`
    RETURN count(r) as updatedCount
    """

    try:
        with driver.session() as session:
            result = session.run(query)
            updated_count = result.single()[0]
            print(f"Propiedad '{propiedad}' eliminada de {updated_count} relaciones.")
    except Exception as e:
        print(f"Error al intentar eliminar la propiedad: {e}")


In [6]:


def update_property_in_relationship(driver):
    print("Actualizar una propiedad en una relación específica")
    
    # Solicitar los datos necesarios del usuario
    nodoA = input("Ingrese la etiqueta del nodo de salida (ej. Person): ").strip()
    llaveNodoA = input("Ingrese la clave del nodo de salida (ej. id): ").strip()
    valorLlaveNodoA = input("Ingrese el valor de la clave del nodo de salida: ").strip()
    nodoB = input("Ingrese la etiqueta del nodo de llegada (ej. Movie): ").strip()
    llaveNodoB = input("Ingrese la clave del nodo de llegada (ej. id): ").strip()
    valorLlaveNodoB = input("Ingrese el valor de la clave del nodo de llegada: ").strip()
    nombreRelacion = input("Ingrese el tipo de relación (ej. ACTED_IN): ").strip()
    nombrePropiedad = input("Ingrese el nombre de la propiedad a actualizar (ej. role): ").strip()
    valorPropiedad = input("Ingrese el nuevo valor para la propiedad: ").strip()

    # Construir y ejecutar la consulta Cypher utilizando interpolación segura para los nombres y parámetros para los valores
    query = f"""
    MATCH (a:`{nodoA}`)-[r:`{nombreRelacion}`]->(b:`{nodoB}`)
    WHERE a.`{llaveNodoA}` = $valorLlaveNodoA AND b.`{llaveNodoB}` = $valorLlaveNodoB
    SET r.`{nombrePropiedad}` = $valorPropiedad
    RETURN a, r, b
    """

    try:
        with driver.session() as session:
            result = session.run(query, valorLlaveNodoA=valorLlaveNodoA, valorLlaveNodoB=valorLlaveNodoB, valorPropiedad=valorPropiedad)
            for record in result:
                print("Actualización realizada:", record["a"], record["r"], record["b"])
    except Exception as e:
        print(f"Error al intentar actualizar la propiedad: {e}")


In [7]:

def update_property_based_on_filter(driver):
    print("Actualizar una propiedad en una relación basada en un filtro específico")
    
    # Solicitar y validar la entrada para las etiquetas de los nodos y la relación
    nodoA = input("Ingrese la etiqueta del nodo de salida (ej. Person): ").strip()
    nodoB = input("Ingrese la etiqueta del nodo de llegada (ej. Movie): ").strip()
    nombreRelacion = input("Ingrese el tipo de relación (ej. ACTED_IN): ").strip()
    nombrePropiedad = input("Ingrese el nombre de la propiedad a actualizar (ej. role): ").strip()
    valorPropiedad = input("Ingrese el nuevo valor para la propiedad: ").strip()
    
    variableAFiltrar = input("Ingrese la variable por la cual filtrar (ej. age, rating): ").strip()
    operador = input("Ingrese el operador de comparación (ej. =, >, <, >=, <=, <>): ").strip()
    valorAFiltrar = input("Ingrese el valor del filtro para la variable especificada: ").strip()

    # Construir y ejecutar la consulta Cypher utilizando interpolación segura y parámetros
    query = f"""
    MATCH (a:`{nodoA}`)-[r:`{nombreRelacion}`]->(b:`{nodoB}`)
    WHERE a.`{variableAFiltrar}` {operador} $valorAFiltrar
    SET r.`{nombrePropiedad}` = $valorPropiedad
    RETURN a, r, b
    """

    try:
        with driver.session() as session:
            result = session.run(query, {'valorAFiltrar': valorAFiltrar, 'valorPropiedad': valorPropiedad})
            for record in result:
                print(f"Actualización realizada en la propiedad '{nombrePropiedad}' de los nodos relacionados:", record["a"], record["r"], record["b"])
    except Exception as e:
        print(f"Error al intentar actualizar la propiedad basada en el filtro: {e}")


In [8]:

def remove_property_from_relationship(driver):
    print("Eliminar una propiedad específica de una relación")

    # Solicitar los datos necesarios del usuario
    nodoA = input("Ingrese la etiqueta del nodo de salida (ej. Person): ").strip()
    llaveNodoA = input("Ingrese la clave del nodo de salida (ej. id): ").strip()
    valorLlaveNodoA = input("Ingrese el valor de la clave del nodo de salida: ").strip()
    nodoB = input("Ingrese la etiqueta del nodo de llegada (ej. Movie): ").strip()
    llaveNodoB = input("Ingrese la clave del nodo de llegada (ej. id): ").strip()
    valorLlaveNodoB = input("Ingrese el valor de la clave del nodo de llegada: ").strip()
    nombreRelacion = input("Ingrese el tipo de relación (ej. ACTED_IN): ").strip()
    nombrePropiedad = input("Ingrese el nombre de la propiedad que desea eliminar (ej. role): ").strip()

    # Construir y ejecutar la consulta Cypher utilizando interpolación segura para los nombres y parámetros para los valores
    query = f"""
    MATCH (a:`{nodoA}`)-[r:`{nombreRelacion}`]->(b:`{nodoB}`)
    WHERE a.`{llaveNodoA}` = $valorLlaveNodoA AND b.`{llaveNodoB}` = $valorLlaveNodoB
    REMOVE r.`{nombrePropiedad}`
    RETURN a, r, b
    """

    try:
        with driver.session() as session:
            result = session.run(query, valorLlaveNodoA=valorLlaveNodoA, valorLlaveNodoB=valorLlaveNodoB)
            updated_count = 0
            for record in result:
                updated_count += 1
                print("Propiedad eliminada:", record["a"], record["r"], record["b"])
            print(f"Total de relaciones actualizadas: {updated_count}")
    except Exception as e:
        print(f"Error al intentar eliminar la propiedad: {e}")


In [9]:

def add_property_to_relationship(driver):
    print("Agregar una nueva propiedad a una relación específica")

    # Solicitar los datos necesarios del usuario
    nodoA = input("Ingrese la etiqueta del nodo de salida (ej. Person): ").strip()
    llaveNodoA = input("Ingrese la clave del nodo de salida (ej. id): ").strip()
    valorLlaveNodoA = input("Ingrese el valor de la clave del nodo de salida: ").strip()
    nodoB = input("Ingrese la etiqueta del nodo de llegada (ej. Movie): ").strip()
    llaveNodoB = input("Ingrese la clave del nodo de llegada (ej. id): ").strip()
    valorLlaveNodoB = input("Ingrese el valor de la clave del nodo de llegada: ").strip()
    nombreRelacion = input("Ingrese el tipo de relación (ej. ACTED_IN): ").strip()
    nombrePropiedad = input("Ingrese el nombre de la nueva propiedad que desea agregar (ej. role): ").strip()
    valorPropiedad = input("Ingrese el valor de la nueva propiedad: ").strip()

    # Construir y ejecutar la consulta Cypher utilizando interpolación segura para los nombres y parámetros para los valores
    query = f"""
    MATCH (a:`{nodoA}`)-[r:`{nombreRelacion}`]->(b:`{nodoB}`)
    WHERE a.`{llaveNodoA}` = $valorLlaveNodoA AND b.`{llaveNodoB}` = $valorLlaveNodoB
    SET r.`{nombrePropiedad}` = $valorPropiedad
    RETURN a, r, b
    """

    try:
        with driver.session() as session:
            result = session.run(query, {'valorLlaveNodoA': valorLlaveNodoA, 'valorLlaveNodoB': valorLlaveNodoB, 'valorPropiedad': valorPropiedad})
            updated_count = 0
            for record in result:
                updated_count += 1
                print("Nueva propiedad agregada:", record["a"], record["r"], record["b"])
            print(f"Total de relaciones actualizadas: {updated_count}")
    except Exception as e:
        print(f"Error al intentar agregar la nueva propiedad: {e}")



In [27]:
def inputs_relaciones():
    respuesta=gestion_Relaciones()

    if respuesta==1:
        add_property_to_relationship(driver)

    if respuesta==2:
        add_empty_property_to_relationship(driver)

    if respuesta==3:
        update_property_in_relationship(driver)

    if respuesta==4:
        update_property_based_on_filter(driver)

    if respuesta==5:
        remove_property_from_relationship(driver)

    if respuesta==6:
        remove_property_from_relationshipGENERAL(driver)


inputs_relaciones()

Que opcion desea?
1) Agregar propiedad a relacion especifica
2) Agregar propiedad a multiples relaciones
3) Actualizacion de propiedad de relacion especifica
4) Actualizacion de multiples propiedades
5) Eliminacion de una propiedad de relacion especifica
6) Eliminacion de varios propiedades
7) Salir
Eliminar una propiedad de una relación específica
Propiedad 'Reception' eliminada de 68434 relaciones.
